In [1]:
import Analysis
import matplotlib as mpl
from matplotlib import pyplot as plt
import numpy as np
%matplotlib

Using matplotlib backend: TkAgg


In [21]:
from scipy.optimize import curve_fit

In [20]:
def search(data, t):
    seq = data
    min = 0
    max = len(seq) - 1
    while True:        
        m = (min + max) // 2
        if max < min:
            if np.abs(t-seq[m]) < np.abs(t-seq[m-1]):          
                return m
            else:
                return m-1        
        if seq[m] < t:
            min = m + 1
        elif seq[m] > t:
            max = m - 1
        else:
            return m

In [27]:
def data_partition(data_x,data_y,sections):
    x = list()
    y = list()
    for i in sections:
        i_min = search(data_x,i[0])
        i_max = search(data_x,i[1])+1
        x.extend(data_x[i_min:i_max])
        y.extend(data_y[i_min:i_max])
    return np.array(x), np.array(y)

In [19]:
def func(x,freq,amp,phase_l,height,slope,v0):
    return height + amp * np.sin(freq * x + phase_l) + slope * (x - v0)
    
    
def gen_func(fit):
    def fitting_func(x, phase_r):
        return func(x,fit[0],fit[1],phase_r,fit[3],fit[4],fit[5])
    return fitting_func


def make_fit(popt,popt2,left,right):
    d_phi = ((popt2[0] - popt[2])/np.pi)%2
    return Analysis.Fit([popt[0],popt[1],popt2[0],popt[2],popt[3],popt[4],1,left,right,d_phi,1,popt[5]])

In [2]:
run_2014 = Analysis.generate_2014()
run_2011 = Analysis.generate_2011()

In [5]:
for i in run_2014.prune_data:
    plt.figure()
    i.plot()
    for j in i.fits:
        j.plot(False)
    

In [52]:
def regression(filenum,edge_list):

    x_temp, y_temp = data_partition(run_2014.prune_data[filenum].x,run_2014.prune_data[filenum].y,edge_list)
    
    if len(run_2014.prune_data[filenum].fits) > 0:

        guess = [run_2014.prune_data[filenum].fits[0].freq,run_2014.prune_data[filenum].fits[0].amp,run_2014.prune_data[filenum].fits[0].phase_l,run_2014.prune_data[filenum].fits[0].height,run_2014.prune_data[filenum].fits[0].slope,run_2014.prune_data[filenum].fits[0].v0]
        popt, pcov = curve_fit(func, x_temp, y_temp, p0 = guess)
    
    else:
        popt, pcov = curve_fit(func, x_temp, y_temp)

    return popt

In [57]:
def regression_2(filenum,popt,edge_list):
    slip_func = gen_func(popt)
    x_temp, y_temp = data_partition(run_2014.prune_data[filenum].x,run_2014.prune_data[filenum].y,edge_list)
    temp = (popt[2]%(2*np.pi))
    guess = temp - 0.25 * np.pi
    popt2, pcov2 = curve_fit(slip_func, x_temp, y_temp,p0=[guess],bounds = ([-2*np.pi],[2*np.pi]))
    return slip_func, popt2

In [49]:
filenum = 12

In [65]:
run_2014.prune_data[filenum].plot()
for i in run_2014.prune_data[filenum].fits:
    i.plot(False)

In [40]:
run_2014.prune_data[filenum].plot_fit(0)

In [55]:
out = regression(filenum,[[-30.719,-21.09]])

In [58]:
run_2014.prune_data[filenum].plot()
plt.plot(run_2014.prune_data[filenum].x,func(run_2014.prune_data[filenum].x,*out),lw=3)

In [59]:
f, out2 = regression_2(filenum, out, [[-27.1543,-26.8537],[-26.7075,-26.5547],[-24.3996,-24.3357]])

In [60]:
run_2014.prune_data[filenum].plot()
plt.plot(run_2014.prune_data[filenum].x,f(run_2014.prune_data[filenum].x,*out2),lw=3)

In [63]:
2 - (out[2]-out2[0])/np.pi%2

0.23075031465534224

In [67]:
run_2014.prune_data[filenum].plot()
plt.plot(run_2014.prune_data[filenum].x,func(run_2014.prune_data[filenum].x,*out),lw=3,c='k')
plt.plot(run_2014.prune_data[filenum].x,f(run_2014.prune_data[filenum].x,*out2),lw=3,c='r')

In [68]:
f2, out3 = regression_2(filenum, out, [[-30.2696,-30.2331],[-29.9777,-29.901],[-29.5775,-294785]])

In [100]:
filenum = 0
run_2014.prune_data[filenum].plot()
for i in run_2014.prune_data[filenum].fits:
    i.plot(False)

In [93]:
run_2014.prune_data[filenum].plot()